# Experimental Data Analysis on 3-D Printed Frames
Written by: Golnar Gharooni Fard 

Last modified: 2022-10-04

This notebook is in support of the paper "Honeycomb crystallography in comb formation under geometric frustrations", showing automatic cell detection and centroid extraction using a sample of experimental data. 

## Import Packags

In [ ]:
import csv
import cv2
import math
import os 

import numpy as np
from numpy import inf, asarray
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm, patches
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

#Image Processing
import PIL
from PIL import Image, ImageDraw
from skimage.filters import threshold_otsu, threshold_niblack, threshold_sauvola, sobel
from skimage import io
from skimage.color import rgb2gray, label2rgb
from skimage.segmentation import felzenszwalb, mark_boundaries, clear_border
from skimage.util import img_as_float
from skimage.measure import label, regionprops
from skimage.morphology import closing, square

from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull
from scipy.spatial.distance import euclidean

plt.rcParams['figure.dpi'] = 300

Image info - This section should be edited for every single image 

In [ ]:
input_folder = '../data' # change to match the address of your I/O folder
plot_folder = '../plots/'

if not os.path.isdir(plot_folder):
    os.mkdir(plot_folder)

# Use the data samples in the input folder 
# get the following info from the image name "file_name_date.png"
file_name = '2D'
date = '6-18'

im = Image.open('{}/{}_{}.png'.format(input_folder, file_name, date))
im_gray = cv2.imread('{}/{}_{}.png'.format(input_folder, file_name, date), 0)

fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(14,10), sharex=True, sharey=True)

ax1.imshow(im)
ax1.set_title('raw image')

# Apply Histogram Equalization to the image
hist,bins = np.histogram(im_gray.flatten(),256,[0,256])
cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/cdf.max()

#find minimum histogram value and apply histogram equalization
equ = cv2.equalizeHist(im_gray)
ax2.imshow(equ, cmap = plt.cm.gray)
ax2.set_title('gray-scaled + contrast adjusted')

cv2.imwrite('{}/{}_{}_equalized.jpg'.format(plot_folder, file_name, date), equ)

## Automatic detection of individual cells

In [ ]:
def find_components(img, lower_bound, upper_bound, plot_dir='../plots/'):
    
    """
    Finds connected components/regions is an image.

    Parameters
    ----------
    img: A crop of the desired frame
    
    lower_bound: smallest cell size that should be detected (should be in a floating point format)
    
    upper bound: largest cell size that should be detected (should be in a floating point format)
    
    plot_dir: Directory where plots are saved. 
    
    Returns
    -------
    dim: Reduced image dimentions  
    
    label_img: image with the regions detected
    
    images_label_overlay: colored labeled image with colors based on region size
        
    cell_positions: a list of coordiantes (y,x) containing the centroids of each region.
        
    """
    img = (img[::2, ::2, :3])

    dim = img.shape
    print ('Modified image dimensions: ', dim)

    segments_fz = felzenszwalb(img, scale=100, sigma=0.5, min_size=700)
    gradient = sobel(rgb2gray(img))
    
    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
    ax1.imshow(mark_boundaries(img, segments_fz))
    ax1.set_title("after Felzenszwalbs algorithm", fontsize =6)
    
    images = mark_boundaries(img, segments_fz)
    images = rgb2gray(images)
    thresh = threshold_otsu(images)
    closed_img = closing(images > thresh, square(3))
    bw = 255 - closed_img 

    # remove artifacts connected to image border
    cleared = clear_border(bw)

    # label image regions
    label_image = label(cleared)
    cell_sizes = []    
    cell_colors = []
    cell_positions = []
    cell_circularity = []
    cell_width_ratio = []
    cell_count = 1

    for region in regionprops(label_image, coordinates='xy'):

        # select regions with large enough areas
        if (region.area > lower_bound and region.area < upper_bound): 

            # draw rectangle around segmented cells
            minr, minc, maxr, maxc = region.bbox 
            cell_sizes.append(region.area)
            sizes = ((region.area) / upper_bound) # scaling cell sizes  
                                                
            cmap = plt.cm.get_cmap('winter')
            rgba = cmap(sizes)
            cell_colors.append(rgba)

            cell_positions.append(region.centroid)
            cell_circularity.append(region.eccentricity)
            cell_width_ratio.append((maxr / maxc))

            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='red', linewidth=1)
            ax2.add_patch(rect)
            cell_count += 1

        else:
            
            # color the background/undetected areas as black
            cell_colors.append((1, 1, 1))

    images_label_overlay = label2rgb(label_image, bg_label=0, colors=cell_colors)
    img_seg = ax2.imshow(images_label_overlay, interpolation='nearest', cmap=cmap)  
    ax2.set_title('after automatic cell detection', fontsize =6)
    fig.colorbar(img_seg, fraction=0.05, aspect=15 , ax=ax2)
    
    plt.imsave ('{}/{}_{}_segmented.png'
                .format(plot_folder, file_name, date), images_label_overlay)
    
    print ("Number of cells automatically detected = {} \n".format(len(cell_sizes)))    
    print ('Minimum cell area detected = {} pixels'.format(np.min(cell_sizes)))
    print ('Maximum cell area detected = {} pixels'.format(np.max(cell_sizes)))
    return (dim, label_image, images_label_overlay, cell_positions ,cell_sizes)

    plt.tight_layout()

In [ ]:
# The following input values are experimentally verified
lower_bound = 400.0
upper_bound = 1500.0

RGB_img = img_as_float(io.imread('{}/{}_{}.png'.format(input_folder, file_name, date)))
d, labeled_img , overlay_img, centroids, volumes = find_components(RGB_img, lower_bound, upper_bound)


## Voronoi Reconstruction

In [ ]:

fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize = [12,16])

ax1.imshow(RGB_img[::2, ::2, :3], cmap=plt.cm.gray)

x_lim = d[-2]
y_lim = d[-3]

# plot the centroids and boxes around individual cells
for props in regionprops(labeled_img, coordinates ='xy'):
            
        if (props.area > lower_bound and props.area < upper_bound):
            
            y0, x0 = props.centroid
            orientation = props.orientation

            x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
            y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
            x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
            y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length

            ax1.plot(x0, y0, '.r', markersize=1)

            minr, minc, maxr, maxc = props.bbox
            bx = (minc, maxc, maxc, minc, minc)
            by = (minr, minr, maxr, maxr, minr)
            ax1.plot(bx, by, '-', color = 'white', linewidth=1)
            
# extract centroids and create a point cloud
points = zip()
points_list = list(points)

ys = [x[0] for x in centroids]
xs = [x[1] for x in centroids]

points = zip(xs, ys)
points_list = list(points)

ax2.plot (xs, ys, '.r', markersize = 2)

# plot the Voronoi reconstruction based on the point cloud data
vor = Voronoi(points_list)
voronoi_plot_2d(vor, show_vertices=False, line_colors='k', line_width=2, line_alpha=0.6, point_size=2, ax=ax3)

# colorize the Voronoi diagram based on the number of sides/neighbors of each cell
norm = mpl.colors.Normalize(vmin=4, vmax=8, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.seismic)

voronoi_plot_2d(vor, show_points=True, show_vertices=False, s=0.2, point_size=1,ax=ax4)

for region in vor.regions:
    if not -1 in region:
        polygon = [vor.vertices[i] for i in region]
        vor_diag = ax4.fill(*zip(*polygon), color=mapper.to_rgba(len(region)))

# overlay the Voronoi recunstruction on the original image as background
background_file = '{}/{}_{}.png'.format(input_folder, file_name, date)

im = Image.open(background_file)  
w, h = im.size
newsize = (x_lim, y_lim) 
new_im = im.resize(newsize)

fig, ax5 = plt.subplots ()
voronoi_plot_2d(vor, show_vertices=False, line_colors='red',
                line_width=1, line_alpha=0.8, point_size=2, ax=ax5)

ax5.imshow(new_im, interpolation='none')

ax5.set_aspect('equal')
ax4.set_aspect('equal')
ax3.set_aspect('equal')
ax2.set_aspect('equal')

ax1.axis((0, x_lim, y_lim, 0))
ax5.axis((0, x_lim, y_lim, 0))

## Save Centroid Coordinates as CSV data

Check the overlay image carefully and save the centroid data in a CSV file

In [ ]:
csv_folder = "../centroids_coordinates/"

centroid_info = pd.DataFrame({'cx':[x[0] for x in points_list],'cy':[x[1] for x in points_list]})
centroid_info.to_csv('{}/{}_{}_cell_centroids.csv'.format(csv_folder, file_name, date))

centroid_info.head()
